<a href="https://colab.research.google.com/github/mohmadAyman75/Face-classification/blob/main/startup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from google.colab import files
upload = files.upload()

Saving Aligned_Faces.zip to Aligned_Faces.zip


In [30]:
import os
import cv2
import numpy as np
import tempfile
import zipfile
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [31]:
DATASET_DIR = r"Aligned_Faces"   # المسار الي بيبقي فيه الداتا
TARGET_IMAGES = 5     # هنا عدد الصور اتحكم فيها براختك
IMG_SIZE = 160

In [32]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.15,
    brightness_range=[0.7, 1.3],
    horizontal_flip=True
)

In [33]:
def generate_augmented_images(image, save_dir, start_index, n):
    image = np.expand_dims(image, axis=0)
    gen = datagen.flow(image, batch_size=1)

    for i in range(n):
        aug_img = next(gen)[0].astype("uint8")
        out_path = os.path.join(save_dir, f"aug_{start_index + i}.jpg")
        cv2.imwrite(out_path, aug_img)

In [ ]:
ZIP_PATH = "Aligned_Faces.zip"
TARGET_IMAGES = 50
IMG_SIZE = 224

def generate_augmented_images(image, save_dir, start_index, n):
    image = np.expand_dims(image, axis=0)
    gen = datagen.flow(image, batch_size=1)

    for i in range(n):
        aug_img = next(gen)[0].astype("uint8")
        out_path = os.path.join(save_dir, f"aug_{start_index + i}.jpg")
        cv2.imwrite(out_path, aug_img)

temp_dir = tempfile.mkdtemp()
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(temp_dir)

DATASET_DIR = os.path.join(temp_dir, "Aligned_Faces")

for person in os.listdir(DATASET_DIR):
    person_folder = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(person_folder):
        continue
    images_in_folder = [f for f in os.listdir(person_folder) if f.lower().endswith(('.jpg', '.png'))]
    img_count = len(images_in_folder)
    print(f"[Processing] {person} → {img_count} images")
    if img_count >= TARGET_IMAGES:
        continue
    missing = TARGET_IMAGES - img_count
    # Check if there's at least one image to augment from
    if images_in_folder:
        base_img_path = os.path.join(person_folder, images_in_folder[0])
        img = cv2.imread(base_img_path)
        if img is not None:
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            generate_augmented_images(img, person_folder, img_count, missing)
            print(f"Added {missing} augment\n")
        else:
            print(f"Could not read image {base_img_path}, skipping augmentation for {person}\n")
    else:
        print(f"No images found for {person}, cannot augment.\n")

print("process done")

shutil.make_archive("Aligned_faces_augmented", 'zip', DATASET_DIR)
shutil.rmtree(temp_dir)

[Processing] Eva_Amurri → 1 images
Added 49 augment

[Processing] Desiree_McKenzie → 1 images
Added 49 augment

[Processing] Grace_Dodd → 1 images
Added 49 augment

[Processing] Naoto_Kan → 4 images
Added 46 augment

[Processing] Elena_de_Chavez → 1 images
Added 49 augment

[Processing] David_Collenette → 1 images
Added 49 augment

[Processing] Jimmy_Jimenez → 1 images
Added 49 augment

[Processing] Wang_Yingfan → 2 images
Added 48 augment

[Processing] Mukesh_Ambani → 3 images
Added 47 augment

[Processing] Yasser_Arafat → 4 images
Added 46 augment

[Processing] Chris_Cookson → 1 images
Added 49 augment

[Processing] Hikmat_al-Azzawi → 1 images
Added 49 augment

[Processing] Kirk_Doerger → 1 images
Added 49 augment

[Processing] Gina_Centrello → 1 images
Added 49 augment

[Processing] Carla_Gugino → 1 images
Added 49 augment

[Processing] Kim_Cattrall → 1 images
Added 49 augment

[Processing] Joe_Lieberman → 8 images
Added 42 augment

[Processing] Severino_Antinori → 1 images
Added 49

In [52]:
from collections import Counter

label_counts = Counter(labels_encoded)
min_count = min(label_counts.values())

print(f"Minimum number of samples per class: {min_count}")

if min_count >= 2:
    print("All classes have at least 2 samples. You can now re-run the train_test_split cell.")
else:
    print("Some classes still have less than 2 samples. Please investigate the augmentation process.")

Minimum number of samples per class: 1
Some classes still have less than 2 samples. Please investigate the augmentation process.


In [35]:
import os
import cv2
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [36]:
DATASET_DIR = r"Aligned_Faces"   # المسار الي بيبقي فيه الداتا
IMG_SIZE = 160

In [40]:
temp_dir = tempfile.mkdtemp()
with zipfile.ZipFile("Aligned_Faces.zip", "r") as z:
    z.extractall(temp_dir)

DATASET_DIR = os.path.join(temp_dir, "Aligned_Faces")

images = []
labels = []

for person in os.listdir(DATASET_DIR):
    person_folder = os.path.join(DATASET_DIR, person)
    if not os.path.isdir(person_folder):
        continue

    for img_file in os.listdir(person_folder):
        if img_file.lower().endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(person_folder, img_file)

            img = cv2.imread(img_path)
            if img is None:
                continue

            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))

            images.append(img)
            labels.append(person)


In [41]:
images = np.array(images) ### عشان تبقي ف array
labels = np.array(labels)

print("Images shape:", images.shape)
print("Labels shape:", labels.shape)

Images shape: (11661, 160, 160, 3)
Labels shape: (11661,)


In [42]:
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

print("Encoded labels:", labels_encoded[:10])
print("Classes:", le.classes_)

Encoded labels: [1497 1212 1773 3667 3667 3667 3667 1378 1115 2385]
Classes: ['AJ_Cook' 'AJ_Lamas' 'Aaron_Guiel' ... 'Zumrati_Juma' 'Zurab_Tsereteli'
 'Zydrunas_Ilgauskas']


In [53]:
X_train, X_test, y_train, y_test = train_test_split(
    images, labels_encoded, test_size=0.2, random_state=42, stratify=labels_encoded
)

print("Train:", X_train.shape, "Test:", X_test.shape)

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.